# **The Effects of Automobile Dependency in the United States**
## Contributors: Nicholas Breymaier and Zachary Osborne
Nicholas UID: 117920871 <br>
Zachary UID: 117941609 <br>
<a href='https://github.com/nicholasbreymaier/nicholasbreymaier.github.io'> Source Code </a>

# Introduction
A place, ranging from a neighborhood-scale to nation-scale geographic area, is referred to as <a href='https://www.planetizen.com/definition/automobile-dependency'>automobile dependent</a>, or car dependent, when the only means by which residents are realistically capable of reaching necessities is by driving a personal automobile. By necessities we mean any locations necessary to live a successful life in modern society, such as institutions of education, jobs, and grocery stores. For the purposes of this analysis, the term 'car' or 'automobile' includes standard sedans and station wagons as well as vans and <a href='https://www.ecfr.gov/current/title-40/chapter-I/subchapter-C/part-86/subpart-A/section-86.082-2'>light-duty trucks</a> such as Sport Utility Vehicles (SUVs) and pickup trucks.

This project serves as an analysis of some of the conseqeuences of and alternatives to car dependency in the United States over time for the purpose of raising awareness, but is by no means meant to be a comprehensive review of the subject. For readers who are interested in learning more about the causes of car dependency and how to fix it, we strongly encourage that you do more research on the subject; the "Further Information for Interested Readers" section at the bottom of this webpage may serve as a good place to start.

We will focus primarily on automobile dependency in the United States, but data from other countries will be used for comparative purposes.

## The Importance of Automobile Dependency in the United States
Over <a href='https://www.forbes.com/advisor/car-insurance/car-ownership-statistics'>90%</a> of households in the United States (US) owned at least one car in 2021, with more than <a href='https://www.forbes.com/advisor/car-insurance/car-ownership-statistics/'>59%</a> owning two or more cars owing to the commonness of married couples having seperate jobs. In 2015, over <a href='https://www.bts.gov/content/commute-mode-share-2015'>80%</a> of commutes were taken by car, as were <a href="https://www.epa.gov/greenvehicles/what-if-more-people-bought-groceries-online-instead-driving-store">88%</a> of grocery store trips. As of 2019, over <a href='https://www.bloomberg.com/news/articles/2019-05-07/how-distance-to-school-affects-student-well-being#:~:text=Today%2C%20nearly%2060%20percent%20of,especially%20on%20sleep%20and%20exercise.'>60%</a> of students in primary and secondary education are driven to school. These trends arise because the vast majority of Americans simply have no other choice but to reach their destinations by driving, whether this be due to distance<a href='https://www.nrc.gov/docs/ML1006/ML100621425.pdf'>—the average commute is 15 miles—</a>or to lack of safety, which we address below. In both of these cases, there is a severe lack of safe, long distance alternatives such as <a href='https://www.apta.com/news-publications/public-transportation-facts/'>public transportation</a>. In other words, the United States <a href='https://www.vox.com/22662963/end-driving-obsession-connectivity-zoning-parking'>is car dependent</a>.

This is problematic for <a href ='https://www.strongtowns.org/journal/2015/1/20/the-negative-consequences-of-car-dependency#:~:text=Living%20in%20a%20car%20dependent,)%20and%20indirect%20(taxes.)'>many reasons</a>, the most dire of which are environmental, financial, and mortal—that is, deadly. We are in the middle of the <a href='https://www.who.int/health-topics/climate-change#tab=tab_1'>climate crisis</a> scientists have been warning us of for decades, with a dire and immediate need to halt carbon emissions and find ways to help the ecosphere heal from the damage we've already done. In the midst of this, with transportation being the <a href='https://www.epa.gov/ghgemissions/sources-greenhouse-gas-emissions'>largest contributor</a> to the US's carbon output, the US Department of Transportation is still <a href='https://highways.dot.gov/federal-lands/projects/by-state'>building and widening</a> highways in <a href='https://www.vtpi.org/gentraf.pdf'>futile</a> attempts to relieve incessant conjestion brought about by people having no alternatives to driving, the very construction process of which emits grand quantities of greenhouse gases. This project analyzes the financial and mortal costs of car dependency in the US in more detail below.

## Relevance in the Field of Data Science
Coverage of causes of death by major media institutions in the United States <a href='https://ourworldindata.org/does-the-news-reflect-what-we-die-from'>fails to correlate</a> with what Americans die from, with automobile fatalities and pollution induced illnesses being covered at significantly lower proportions than the proportion of deaths they cause. This is likely due in part to the need for major media companies to "keep up" with the incessant and immediate news which is able to be transmitted via social media, which has lead to companies disproportunately covering eye catching topics like homicide and terrorism to maintain viewership and therefore advertisement revenue. This is part of a larger <a href='https://www.gsb.stanford.edu/insights/media-consolidation-means-less-local-news-more-right-wing-slant'>trend</a> wherein media consolidation into the hands of large corporations causes news stations to prioritize national news as opposed to local news, and often leads to right-wing political slant in coverage. Moreover, major US news corporations increasingly fail to critically examine institutions in relation to societal problems, opting to either not discuss them at all or to atomize important events which are symptoms of larger problems, as may be seen in the coverage of the recent East Palestine <a href='https://en.wikipedia.org/wiki/2023_Ohio_train_derailment'>train derailment</a>. Relying on the media is therefore not an effective way to build public awareness about systemic, societal problems.

Data Science offers a promising alternative medium of news coverage as opposed to major media corporations and news transmitted via social media because it is significantly <a href='https://fivethirtyeight.com/features/what-the-fox-knows/'>less prone to outlier bias</a>. It is capable of analyzing long-term macro-level trends that would take the culmination of thousands of traditional articles to reveal, much less communicate, and may be seen as more democratic than the privately owned media corporations which currently dominate the US news landscape. Anyone with the requisite knowledge, a computer, and internet access can use data science to bring issues to light. This project hopes to demonstrate that data science can be used to generate awareness about important yet overlooked societal issues by using it to do so with car dependency.

# Step 1: Data Collection
Thankfully, the US National Highway Traffic Safety Administration (NHTSA) keeps a nicely detailed <a href='https://www-fars.nhtsa.dot.gov/Main/index.aspx'>data table</a> displayed on its website concerning traffic fatalities in recent years, which serves as a good starting point for discussing the problem of car dependency.

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:

# # Use the python requests library to extract the data from the website.
# web_extract = requests.get(
#     'https://www-fars.nhtsa.dot.gov/Main/index.aspx').text

# # Organise and make legible the extracted html using the BeautifulSoup library.
# web_extract = bs(web_extract)
# web_extract.prettify()

# loci = web_extract.find('table')

# # Extract the html tables into pandas dataframes. Tables 1–9 were
# # stylistic elements in the website.
# nhtsa_nat_stats = pd.read_html(str(loci))[10]

nhtsa_nat_stats = pd.read_feather('nhtsa_nat_stats.feather')
nhtsa_nat_stats.head()

https://www.bts.gov/content/transit-profile-0

only do rough selection of categories of data we do/dont want here, do fine-grained removal in data processing (for all 3 profiles)

In [ ]:
blank_headers = [2, 45, 73, 140]
undesired_data = list(range(27, 45)) + list(range(46, 73)) + list(range(74, 92)) + list(range(101, 140)) + list(range(159, 168))

transit_data = pd.read_excel(
    'table_transit_profile_032123.xlsx', 
    header=1, 
    index_col=0, 
    nrows=165,
    skiprows=blank_headers+undesired_data,
    usecols=[0]+list(range(5, 31))
)

transit_data.head()

https://www.bts.gov/content/highway-profile

In [ ]:
blank_headers = [2, 24, 67, 86]
undesired_data = list(range(25, 67)) + list(range(83, 86)) + [89]

highway_data = pd.read_excel(
    'table_highway_profile_072322.xlsx', 
    header=1, 
    index_col=0, 
    nrows=87,
    skiprows=blank_headers+undesired_data,
    usecols=[0]+list(range(5, 31))
)

highway_data.head()

https://www.bts.gov/content/automobile-profile

In [ ]:
blank_headers = [2, 14, 26, 72]
undesired_data = [13] + list(range(15, 26)) + list(range(27, 52)) + list(range(56, 72)) + list(range(80, 105))

automobile_data = pd.read_excel(
    'table_automobile_profile_042522.xlsx', 
    header=1, 
    index_col=0, 
    nrows=102,
    skiprows=blank_headers+undesired_data,
    usecols=[0]+list(range(5, 31))
)

automobile_data.head()

# Step 2: Data Processing
The NHTSA crash fatality data we extracted is far from tidy, so we shall clean and organise it here. 

Firstly, this DataFrame holds features in its rows and observations in its columns. Obervations are, in this case, years. We therefore transpose the dataframe to make the rows observations and the columns features.

In [ ]:
# We will want these to be the column labels after transposing the DataFrame.
nhtsa_nat_stats = nhtsa_nat_stats.set_index('Unnamed: 0')

# Transpose the DataFrame.
nhtsa_nat_stats = nhtsa_nat_stats.transpose()

# Rename index column to 'Year' and sort data chronologically.
nhtsa_nat_stats.index.name = 'Year'
nhtsa_nat_stats = nhtsa_nat_stats.sort_index()

nhtsa_nat_stats.columns.name = None

nhtsa_nat_stats.head()

The original data table groups its features into various categories via extra, empty features which essentially function as "headers". This is a very messy way to do this, and furthermore having the categorization within the table is unnecessary. Here, we remove the unnecessary columns and rename the rest for clarity and to preserve information which would otherwise have been lost during the purging of the headers.

In [ ]:
# Drop empty header columns.
columns_to_remove = [
    'Motor Vehicle Traffic Crashes', 
    'Traffic Crash Fatalities',
    'Vehicle Occupants', 
    'Nonmotorists',
    'Other National Statistics', 
    'National Rates: Fatalities'
]

nhtsa_nat_stats.drop(columns=columns_to_remove, inplace=True)

In [ ]:
# Rename remaining columns for clarity 
nhtsa_nat_stats = nhtsa_nat_stats.rename(
    columns={
        'Drivers' : 'Driver Fatalities',
        'Passengers' : 'Passenger Fatalities',
        'Unknown' : 'Unknown Vehicle Occupant Fatalities',
        'Sub Total1' : 'Total Vehicle Occupant Fatalities',
        'Motorcyclists' : 'Motorcyclist Fatalities',
        'Pedestrians' : 'Pedestrian Fatalities',
        'Pedalcyclists' : 'Pedalcyclist Fatalities',
        'Other/ Unknown' : 'Other/Unknown Nonmotorist Fatalities',
        'Sub Total2' : 'Total Nonmotorist Fatalities',
        'Total*' : 'Total Fatalities'
    }
)

nhtsa_nat_stats.head()

In [ ]:
nhtsa_nat_stats.drop(index='1994', inplace=True)

nhtsa_nat_stats.head()

In [ ]:
columns_to_remove = [
    'Fatal Crashes',
    'Passenger Fatalities',
    'Unknown Vehicle Occupant Fatalities',
    'Motorcyclist Fatalities',
    'Pedestrian Fatalities',
    'Pedalcyclist Fatalities',
    'Other/Unknown Nonmotorist Fatalities',
    'Total Fatalities',
    'Vehicle Miles Traveled (Billions)',
    'Fatalities per 100 Million Vehicle Miles Traveled',
    'Fatalities per 100,000 Registered Vehicles',
    'Fatalities per 100,000 Licensed Drivers'
]

nhtsa_nat_stats.drop(columns=columns_to_remove, inplace=True)

nhtsa_nat_stats.head()

### Transit Profile

In [ ]:
transit_data = transit_data.transpose()
transit_data.index.name = 'Year'

transit_data.head()

In [ ]:
transit_data = transit_data.rename(
    columns={
        'Light raila' : 'Light rail',
        'Light raila ' : 'Light rail',
        'Ferryboatb' : 'Ferryboat',
        'Otherc' : 'Other',
        'Operating assistanced, total' : 'Operating assistance, total',
        'Commuter railf' : 'Commuter rail',
        'Injured persons, all modesk' : 'Injured persons, all modes'
    }
)

transit_data.head()

Removing undesired non-headers

In [ ]:
columns_to_remove = [
    'Motor bus',
    'Heavy rail',
    'Light rail',
    'Trolley bus',
    'Demand responsive',
    'Ferryboat',
    'Commuter rail',
    'Other',
    'Other operating revenue',
    'Operating assistance, total',
    'State and local',
    'Federal'
]
transit_data.drop(columns=columns_to_remove, inplace=True)

transit_data.head()

Passenger operating revenues, total (millions of dollars) functions as a header for Operating revenues, total and Passenger fares, total so we have to rename them before we can remove it

In [ ]:
transit_data.rename(
    columns={
        'Operating revenues, total' : 'Operating revenues, total (millions of dollars)',
        'Passenger fares, total' : 'Passenger fares, total (millions of dollars)'
    },
    inplace=True
)

transit_data.drop(columns='Passenger operating revenues, total (millions of dollars)', inplace=True)

transit_data.head()

### Highway Profile

In [ ]:
highway_data = highway_data.transpose()
highway_data.index.name = 'Year'

highway_data.head()

In [ ]:
highway_data = highway_data.rename(
    columns={
        'Highway trust funda' : 'Highway trust fund',
        'Otherb' : 'Other',
        'State highway user tax revenuesc, total (millions of dollars)' : 'State highway user tax revenues, total (millions of dollars)',
        'Other motor fuel receiptsd' : 'Other motor fuel receipts',
        'Other motor vehicle feese' : 'Other motor vehicle fees',
        'Motor carrier taxesf' : 'Motor carrier taxes',
        'Miscellaneous feesg' : 'Miscellaneous fees',
        'Vehicle-miles of travel by functional system (millions), total n' : 'Vehicle-miles of travel by functional system (millions), total',
        'Collectorj' : 'Collector'
    }
)

highway_data.head()

Removing undesired non-headers

In [ ]:
columns_to_remove=[
    'Federal, total',
    'Highway trust fund',
    'Other',
    'State and local, total',
    'State and D.C.',
    'Local',
    'Motor fuel tax',
    'Other motor fuel receipts',
    'Motor vehicle registration fees ',
    'Other motor vehicle fees',
    'Motor carrier taxes',
    'Miscellaneous fees',
    'Rural mileage, total',
    'Interstate',
    'Interstate ',
    'Other freeways and expressways',
    'Other principal arterial',
    'Minor arterial',
    'Collector',
    'Urban mileage, total'
]

highway_data.drop(columns=columns_to_remove, inplace=True)

highway_data.head()

### Automobile Profile

In [ ]:
automobile_data = automobile_data.transpose()
automobile_data.index.name = 'Year'

automobile_data.head()

In [ ]:
automobile_data = automobile_data.rename(
    columns={
        'Personal auto expenditures, totala (millions of dollars)' : 'Personal auto expenditures, total (millions of dollars)',
        'Light duty vehicle, short wheel basec,j' : 'Light duty vehicle, short wheel base',
        'Light duty vehicle, long wheel basee,j' : 'Light duty vehicle, long wheel base',
        'Motorcyclef,i' : 'Motorcycle',
        'Bicycled' : 'Bicycle',
        'Pedestriand' : 'Pedestrian'
    }
)

automobile_data.head()

In [ ]:
columns_to_remove = [
    'New cars',
    'Used cars',
    'Parts and accessories',
    'Gasoline and oil',
    'Maintainence and repair',
    'Other services',
    'Insurance premiums less claims paid',
    'Auto registration fees',
    'Driver\'s license fees',
    'Light duty vehicle, short wheel base',
    'Light duty vehicle, long wheel base',
    'Motorcycle',
    'Passenger car',
    'Light truck',
    'Bicycle',
    'Pedestrian',
    'Other/Unknown'
]

automobile_data.drop(columns=columns_to_remove, inplace=True)

automobile_data.head()

### Combining the data

Confirming index types match

In [ ]:
print(f'Transit index type: {transit_data.index.dtype}')
print(f'Highway index type: {highway_data.index.dtype}')
print(f'Automobile index type: {automobile_data.index.dtype}')
print(f'NHTSA index type: {nhtsa_nat_stats.index.dtype}')

In [ ]:
nhtsa_nat_stats.index = nhtsa_nat_stats.index.astype(transit_data.index.dtype)

nhtsa_nat_stats.index.dtype

In [ ]:
full_df = pd.concat(
    {
        'NHTSA Stats' : nhtsa_nat_stats, 
        'Transit Profile' : transit_data,
        'Highway Profile' : highway_data,
        'Automobile Profile' : automobile_data
    },
    axis=1
)

full_df.head()

In [ ]:
full_df.rename(
    columns={
        'Total Vehicle Occupant Fatalities' : 'Vehicle Occupant Fatalities',
        'Total Nonmotorist Fatalities' : 'Nonmotorist Fatalities',
        'Resident Population (Thousands)' : 'Resident Population (Thousands)',
        'Registered Vehicles (Thousands)' : 'Registered Vehicles (Thousands)',
        'Licensed Drivers (Thousands)' : 'Licensed Drivers (Thousands)',
        'Operating revenues, total (millions of dollars)' : 'Operating Revenues (Millions USD)',
        'Passenger fares, total (millions of dollars)' : 'Passenger Fares (Millions USD)',
        'Operating expenses, total (millions of dollars)' : 'Operating Expenses (Millions USD)',
        'Passenger-miles, total (millions)' : 'Passenger-Miles (Millions)',
        'Fatalities, all modes' : 'Fatalities',
        'Injured persons, all modes' : 'Injured Persons',
        'Government receipts, total (millions of dollars)' : 'Gov Receipts (Millions USD)',
        'Government expenditures, total (millions of dollars)' : 'Gov Expenditures (Millions USD)',
        'State highway user tax revenues, total (millions of dollars)' : 'State Highway User Tax Revenues (Millions USD)',
        'Vehicle-miles of travel by functional system (millions), total' : 'Vehicle-Miles (Millions)',
        'Injured persons' : 'Injured Persons',
        'Personal auto expenditures, total (millions of dollars)' : 'Personal Auto Expenditures (Millions USD)',
        'Number of occupants and nonoccupant fatalities' : 'Fatalities'
    },
    inplace=True
)

full_df.columns.values

##### Missing Data

In [ ]:
full_df[[('Highway Profile', 'Gov Receipts (Millions USD)'), ('Highway Profile', 'Gov Expenditures (Millions USD)')]].tail(10)

In [ ]:
full_df.replace('U', float('NaN'), inplace=True)

full_df[[('Highway Profile', 'Gov Receipts (Millions USD)'), ('Highway Profile', 'Gov Expenditures (Millions USD)')]].tail(10)

##### Feature value types

In [ ]:
full_df = full_df.apply(pd.to_numeric)

##### Final datatable

In [ ]:
full_df

Keeping the below as an example in case we need to do something similar later, remove before submitting

essentially converts header column structure to multi-index structure

In [ ]:
# # preserving information which would otherwise be lost due to removal of the sub-header formatting of
# # the first 15 columns
# trans_p = trans_p.rename(
#     columns={
#         'Operating revenues, total' : 'Total operating revenues',
#         'Passenger fares, total' : 'Total passenger fares',
#         'Other operating revenue' : 'Non-fare operating revenue',
#         'Operating assistance, total' : 'Total operating assistance',
#         'State and local' : 'State and local operating assistance',
#         'Federal' : 'Federal operating assistance'
#     }
# )
# for i in range(3, 11):
#     trans_p.columns.values[i] += ' passenger fares'

# # Renaming columns to remove information which will soon be contained in the top-level columns
# trans_p = trans_p.rename(
#     columns={
#         'Passenger operating revenues, total (millions of dollars)' : 'Total',
#         'Operating expenses, total (millions of dollars)' : 'Total',
#         'Average passenger revenue per passenger-mile, all modes (dollars)' : 'All modes',
#         'Number of vehicles, total' : 'Total',
#         'Vehicle-miles, total (millions)' : 'Total',
#         'Passenger-miles, total (millions)' : 'Total',
#         'Energy consumption, diesel, total (million gallons)' : 'Total',
#         'Energy consumption, other, total (million gallons)' : 'Total',
#         'Energy consumption, electric power, total (million kWh)' : 'Total',
#         'Fatalities, all modes' : 'All modes',
#         'Injured persons, all modes' : 'All modes',
#         'Incidents, all modes' : 'All modes'
#     }
# )

# headers = (
#     ['Passenger Operating Revenues (Millions USD)'] * 15
#     + ['Operating Expenses (Millions USD)'] * 9
#     + ['Avg Passenger Revenue Per Passenger Mile (USD)'] * 9
#     + ['Number of Vehicles'] * 9
#     + ['Vehicle Miles (Millions)'] * 9
#     + ['Passenger Miles (Millions)'] * 9
#     + ['Diesel Energy Consumption (Million Gallons)'] * 9
#     + ['Other Energy Consumption (Million Gallons)'] * 3
#     + ['Electric Energy Consumption (Million kWh)'] * 9
#     + ['Fatalities'] * 9
#     + ['Injured Persons'] * 9
#     + ['Incidents'] * 9
# )

# trans_p.columns = pd.MultiIndex.from_tuples(list(zip(headers, trans_p.columns)))

# trans_p.head()

# Step 3: Exploratory Analysis & Data Visualization

As a part of our exploration and analysis of the data, we will want to visualize it through graphs. To aid in this, we have defined a function for plotting columns of a dataframe against the indices/other columns of the same dataframe below. Note that if one wants to make additional modifications to the plot beyond those of this function, they need only make such modifications prior to calling it. 

In [ ]:
def plot_cols(df, y_cols, title, xlabel=None, ylabel=None, x_col=None):
    """
    Plots the specified columns y_cols of the dataframe df against either the indices of
    the dataframe or optionally a specified column x_col. All plotted lines will be on
    the same graph, with a legend to differentiate them

    If not specified, xlabel will default to x_col/index name and ylabel will default to y_cols[0]
    """

    # y_cols is expected to be a type list, so if the user passes a single column name we
    # want to make sure to convert it to a list of a single item
    y_cols = [y_cols] if type(y_cols) == str else y_cols
    # as explained above, y columns will be graphed against the dataframe indices if no column
    # is specifed for the x-axis
    x = df.index if x_col == None else df[x_col]
    # setting to defaults, as defined above, if necessary
    xlabel = df.columns.name if xlabel == None else xlabel
    ylabel = y_cols[0] if ylabel == None else ylabel

    # plots a line for each column
    for y_col in y_cols:
        plt.plot(x, df[y_col], label=y_col)
    
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.show()

In [ ]:
plt.xticks(range(0, len(nhtsa_nat_stats.index), 5))
plot_cols(nhtsa_nat_stats, 'Fatal Crashes', 'Yearly Fatal Motor Vehicle Crashes in the U.S.')

# Step 4: Modeling and Further Analysis with Machine Learning and Statistics

# Step 5: Data Interpretation and Insight

# Further Information for Interested Persons

There are many advocates and advocacy groups, from local international, fighting to spread awareness about this issue and push for solutions to be implemented. Here a few are listed, as well as some sources of more information about car dependency:
> * <a href='https://en.wikipedia.org/wiki/Car_dependency'>Wikipedia</a> is always a good starting point.
> * <a href='https://www.planetizen.com/definition/automobile-dependency'>Planetizen</a> and <a href='https://www.vtpi.org/tdm/tdm100.htm'>Victoria Transport Poliy Institute</a> both have very good articles defining and detailing some common aspects of car dependency.
> * <a href='https://www.youtube.com/@NotJustBikes/featured'>Not Just Bikes</a>, courtesy of Jason Slaughter, became most popular 'urban planning' channel on YouTube as of 2022. Not Just Bikes explores Dutch urban design and transportation engineering with a focus on comparing it to American and Canadian development. Car dependency is a central topic throughout these videos.
> * <a href='strongtowns.org'>Strong Towns</a> is an American advocacy organisation focused on local governance, city finances, and urban development. Its founder, Charles Marohn, served as a professional traffic engineer, and advocates for reduced car dependency in tandem with zoning reform and road design reform in the US.

# Bibliography
(3) https://nepis.epa.gov/Exe/ZyPDF.cgi?Dockey=P1013L1O.pdf 3 <br>